In [0]:
tabela_job = spark.sql("""
SELECT DISTINCT database, tableName, 
COALESCE(NULLIF(trim(jobId), ''), NULL) AS jobId,
jobName, timestamp FROM dataops_prd.migration.tables_jobs_temp
              """)


In [0]:
%pip install databricks_api

In [0]:
from databricks_api import DatabricksAPI
 
db = DatabricksAPI(
    host="experian-serasa-ecs-labs.cloud.databricks.com",
    token=dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
)

api_jobs = spark.createDataFrame([{'job_id': job['job_id']} for job in db.jobs.list_jobs()['jobs']])
api_jobs = api_jobs.select('job_id').distinct()

In [0]:
null_df = tabela_job.filter("jobId is null")
df = tabela_job.filter("jobId is not null")

In [0]:
join = df.join(
    api_jobs,
    df.jobId == api_jobs.job_id,
    how='left'
)

join_renamed = join.selectExpr(
    "database",
    "tableName",
    "jobId as job_id",
    "jobName as job_name",
    "timestamp"
)

In [0]:
null_df_renamed = null_df.selectExpr(
    "database",
    "tableName",
    "jobId as job_id",
    "jobName as job_name",
    "timestamp"
)

result = join_renamed.unionByName(
    null_df_renamed
)

In [0]:
result.write.mode("overwrite").saveAsTable("dataops_prd.migration.tables_jobs")